#### Author: Roberto Mendoza
#### Analyzing RCT reemployment experiment

Data
In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI).

In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data.

In [ ]:
import Pkg


Pkg.add("DataFrames")
Pkg.add("FilePaths")
Pkg.add("Queryverse")
Pkg.add("GLM")
Pkg.add("StatsModels")
Pkg.add("Combinatorics")
Pkg.add("Iterators")
Pkg.add("CategoricalArrays")
Pkg.add("StatsBase")
Pkg.add("Lasso")
Pkg.add("TypedTables")
Pkg.add("MacroTools")
Pkg.add("NamedArrays")
Pkg.add("DataTables")
Pkg.add("Latexify")
Pkg.add("PrettyTables")
Pkg.add("TexTables")
using GLM, StatsModels, DataTables
using DelimitedFiles, DataFrames, Lasso
using FilePaths
using Queryverse
using StatsModels, Combinatorics
using CategoricalArrays
using StatsBase, Statistics
using TypedTables
using MacroTools
using NamedArrays
using PrettyTables # Dataframe or Datatable to latex
using TexTables # pretty regression table and tex outcome

    Updating registry at `C:\Users\Roberto\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Roberto\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Roberto\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
   Installed FilePaths ─ v0.8.3
    Updating `C:\Users\Roberto\.julia\environments\v1.6\Project.toml`
  [8fc22ac5] + FilePaths v0.8.3
    Updating `C:\Users\Roberto\.julia\environments\v1.6\Manifest.toml`
  [8fc22ac5] + FilePaths v0.8.3
Precompiling project...
  ✓ FilePaths
  1 dependency successfully precompiled in 1 seconds (105 already precompiled, 94 skipped during auto due to previous errors)
   Resolving package versions...
   Installed TableTraitsUtils ────── v1.0.2
   Installed URIParser ───────────── v0.4.1
   Installed ExcelFiles ──────────── v1.0.0
   Installed ReadStat ────────────── v1.1.1
   Installed Nullables ─────────────

In [ ]:
# Loading data

mat, head = readdlm("../../../data/penn_jae.dat", header=true, Float64)
mat
df =DataFrame(mat, vec(head))
describe(df)

In [ ]:
#dimenntions of dataframe 

a = size(df,1)
b =  size(df,2)

In [109]:
# Filter control group and just treatment group number 4

penn = filter(row -> row[:tg] in [4,0], df)

first(penn,20)

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10824.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,2.0
2,10824.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10747.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0
4,10607.0,4.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0
5,10831.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,1.0
6,10845.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,0.0
7,10831.0,0.0,9.0,9.0,1.0,0.0,0.0,0.0,1.0
8,10859.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,1.0
9,10516.0,0.0,15.0,15.0,1.0,0.0,0.0,0.0,0.0


In [110]:
# Treatment group n°4
replace!(penn.tg, 4 => 1)
rename!(penn, "tg" => "T4")


# from float to string
penn[!,:dep] = string.(penn[!,:dep]) 

# dep varaible in categorical format 
penn[!,:dep] = categorical(penn[!,:dep])

describe(penn)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,abdt,10695.4,10404.0,10698.0,10880.0,0,Float64
2,T4,0.342224,0.0,0.0,1.0,0,Float64
3,inuidur1,13.053,1.0,11.0,52.0,0,Float64
4,inuidur2,12.2812,0.0,10.0,52.0,0,Float64
5,female,0.404001,0.0,0.0,1.0,0,Float64
6,black,0.121985,0.0,0.0,1.0,0,Float64
7,hispanic,0.0325554,0.0,0.0,1.0,0,Float64
8,othrace,0.00725632,0.0,0.0,1.0,0,Float64
9,dep,,0.0,,2.0,0,"CategoricalValue{String, UInt32}"


### 2.0 Carry out covariate balance check

In [111]:
    # couples variables combinations 
    combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)

    # combinations without same couple
    expand_exp(args, deg::ConstantTerm) =
        tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

    StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
        apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [112]:
# linear regression

reg1 = @formula(T4 ~ (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)
reg1 = apply_schema(reg1, schema(reg1, penn))

FormulaTerm
Response:
  T4(continuous)
Predictors:
  female(continuous)
  black(continuous)
  othrace(continuous)
  dep(DummyCoding:3→2)
  q2(continuous)
  q3(continuous)
  q4(continuous)
  q5(continuous)
  q6(continuous)
  agelt35(continuous)
  agegt54(continuous)
  durable(continuous)
  lusd(continuous)
  husd(continuous)
  female(continuous) & black(continuous)
  female(continuous) & othrace(continuous)
  female(continuous) & dep(DummyCoding:3→2)
  female(continuous) & q2(continuous)
  female(continuous) & q3(continuous)
  female(continuous) & q4(continuous)
  female(continuous) & q5(continuous)
  female(continuous) & q6(continuous)
  female(continuous) & agelt35(continuous)
  female(continuous) & agegt54(continuous)
  female(continuous) & durable(continuous)
  female(continuous) & lusd(continuous)
  female(continuous) & husd(continuous)
  black(continuous) & othrace(continuous)
  black(continuous) & dep(DummyCoding:3→2)
  black(continuous) & q2(continuous)
  black(continuous) & q3(

In [200]:
m1 = lm(reg1, penn)
table = regtable( "Covariate Balance Check" => m1) # coeficientes, standar error, squared R, N (sample size )

                   | Covariate Balance Check 
                   |           (1)           
---------------------------------------------
       (Intercept) |                  0.321* 
                   |                 (0.167) 
            female |                   0.104 
                   |                 (0.138) 
             black |                   0.038 
                   |                 (0.063) 
           othrace |                  -0.345 
                   |                 (0.294) 
          dep: 1.0 |                  -0.074 
                   |                 (0.218) 
          dep: 2.0 |                  -0.109 
                   |                 (0.165) 
                q2 |                  -0.027 
                   |                 (0.168) 
                q3 |                  -0.006 
                   |                 (0.167) 
                q4 |                   0.043 
                   |                 (0.168) 
                q5 |              

### 3.0 Model specification

In [201]:
# No adjustment (2-sample approach)

ols_cl = lm(@formula(log(inuidur1) ~ T4), penn)

table1 = regtable( "No adjustment model" => ols_cl)   # 

            | No adjustment model 
            |         (1)         
----------------------------------
(Intercept) |            2.057*** 
            |             (0.021) 
         T4 |            -0.085** 
            |             (0.036) 
----------------------------------
          N |                5099 
      $R^2$ |               0.001 


In [202]:
# adding controls
# Omitted dummies: q1, nondurable, muld

reg2 = @formula(log(inuidur1) ~ T4 + (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)
reg2 = apply_schema(reg2, schema(reg2, penn))

ols_cra = lm(reg2, penn)
table2 = regtable("CRA model" => ols_cra)

                   | CRA model 
                   |    (1)    
-------------------------------
       (Intercept) |  2.633*** 
                   |   (0.420) 
                T4 |  -0.080** 
                   |   (0.036) 
            female |    -0.115 
                   |   (0.347) 
             black | -0.441*** 
                   |   (0.158) 
           othrace |    -0.883 
                   |   (0.903) 
          dep: 1.0 |    -0.720 
                   |   (0.550) 
          dep: 2.0 |    -0.041 
                   |   (0.417) 
                q2 |    -0.160 
                   |   (0.423) 
                q3 |    -0.540 
                   |   (0.422) 
                q4 |    -0.433 
                   |   (0.422) 
                q5 |    -0.345 
                   |   (0.420) 
                q6 |    -0.494 
                   |   (0.420) 
           agelt35 |   -0.626* 
                   |   (0.340) 
           agegt54 |    -0.361 
                   |   (0.760) 
        

In [204]:
# demean function
function desv_mean(a)
    A = mean(a, dims = 1)
    M = zeros(Float64, size(X,1), size(X,2))
    for i in 1:size(a,2)
          M[:,i] = a[:,i] .- A[i]
    end
    return M
end    

# Matrix Model & demean
X = StatsModels.modelmatrix(reg1.rhs,penn)
X = desv_mean(X) # matrix format 
 


5099×119 Matrix{Float64}:
 -0.404001  -0.121985  -0.00725632  -0.112179  …  -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632   0.887821      0.945676   -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179  …  -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632   0.887821     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632   0.887821     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179  …  -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0

In [205]:
Y = select(penn, [:inuidur1,:T4]) # select inuidur1 y T4

X = DataFrame(hcat(X, Matrix(select(penn, [:T4])).*X), :auto)  # Joint X, (T4*X)

base = hcat(Y, X) # Joint inuidur1, T4, X y (T4*X)

base.inuidur1 = log.(base.inuidur1)  # log(inuidur1)

terms = term.(names(base)) # term.() let us to get all variables as objects

#interactive regression model

ols_ira  = lm(terms[1] ~ sum(terms[2:end]), base)

table3 = regtable("Interactive model" => ols_ira)

#terms[1] : select first variable. In this case, oucome of interest 
#sum(terms[2:end]) : independent variables as regresors in the linear regression   

            | Interactive model 
            |        (1)        
--------------------------------
(Intercept) |          2.058*** 
            |           (0.021) 
         T4 |          -0.076** 
            |           (0.036) 
         x1 |            -0.666 
            |           (0.443) 
         x2 |         -0.863*** 
            |           (0.269) 
         x3 |            -1.735 
            |           (2.163) 
         x4 |             0.036 
            |           (0.682) 
         x5 |             0.212 
            |           (0.495) 
         x6 |            -0.255 
            |           (0.525) 
         x7 |            -0.621 
            |           (0.523) 
         x8 |            -0.480 
            |           (0.524) 
         x9 |            -0.372 
            |           (0.522) 
        x10 |            -0.677 
            |           (0.519) 
        x11 |            -0.678 
            |           (0.433) 
        x12 |            -0.304 
          

In [206]:
X = StatsModels.modelmatrix(reg2.rhs,penn)
X = desv_mean(X)

X = DataFrame(hcat(X, X[:,1].*X[:,2:end]), :auto)  # Joint X, (T4*X)
Y = select(penn, [:inuidur1]) #select just inuidur1
base = hcat(Y, X)  #joint inuidur1, X, (T4*X)
rename!(base, Dict(:x1 => :T4)) #rename x1 -> T4
base.inuidur1 = log.(base.inuidur1)  # log(inuidur1)

terms = term.(names(base)) # all terms  

#model = terms[1] ~ sum(terms[2:end])

lasso = fit(LassoModel,terms[1] ~ sum(terms[2:end]), base, dropcollinear=false; α = 0.1)
coef(lasso)[2]

table3 = regtable("Lasso adjustment model" => lasso)

┌ Warning: One of the predicators (columns of X) is a constant, so it can not be standardized.
│ To include a constant predicator set standardize = false and intercept = false
└ @ Lasso C:\Users\Roberto Carlos\.julia\packages\Lasso\oArSQ\src\Lasso.jl:360


LoadError: ArgumentError: start and stop must be finite, got NaN and NaN

In [208]:
grouped_table = regtable(   "Group 1"=>(ols_cl),
                            "Group 2"=>(ols_cra),
                            "Group 3"=>(ols_ira))

# append regression outcomes

# to tex format 
to_tex(grouped_table) |> print  

\begin{tabular}{r|c|c|c}
\toprule
                   & \multicolumn{1}{c}{Group 1} & \multicolumn{1}{c}{Group 2} & \multicolumn{1}{c}{Group 3} \\
                   & (1)                         & (2)                         & (3)                         \\ \hline
       (Intercept) &                    2.057*** &                    2.633*** &                    2.058*** \\
                   &                     (0.021) &                     (0.420) &                     (0.021) \\
                T4 &                    -0.085** &                    -0.080** &                    -0.076** \\
                   &                     (0.036) &                     (0.036) &                     (0.036) \\
            female &                             &                      -0.115 &                             \\
                   &                             &                     (0.347) &                             \\
             black &                             &             

In [216]:
# Comparative ATE estimation

table = NamedArray(zeros(4, 5))

table[1,2] = GLM.coeftable(ols_cl).cols[1][2]
table[2,2] = GLM.coeftable(ols_cl).cols[2][2]
table[3,2] = GLM.coeftable(ols_cl).cols[5][2]
table[4,2] = GLM.coeftable(ols_cl).cols[6][2]
table[1,3] = GLM.coeftable(ols_cra).cols[1][2]
table[2,3] = GLM.coeftable(ols_cra).cols[2][2]
table[3,3] = GLM.coeftable(ols_cra).cols[5][2]
table[4,3] = GLM.coeftable(ols_cra).cols[6][2]
table[1,4] = GLM.coeftable(ols_ira).cols[1][2]
table[2,4] = GLM.coeftable(ols_ira).cols[2][2]
table[3,4] = GLM.coeftable(ols_ira).cols[5][2]
table[4,4] = GLM.coeftable(ols_ira).cols[6][2]

T = DataFrame(table, [ :"Outcome", :"CL", :"CRA", :"IRA", :"IRA W Lasso"])  # table to dataframe 
T[!,:Outcome] = string.(T[!,:Outcome])  # string - first column 

T[1,1] = "Estimation"
T[2,1] = "Standar error"
T[3,1] = "Lower bound CI"
T[4,1] = "Upper bound CI"

header = (["Outcome", "CL", "CRA", "IRA", "IRA W Lasso"])

pretty_table(T; backend = Val(:html), header = header, formatters=ft_round(4), alignment=:c)

# Val(:latex) to tex format (Dataframe or DataTable, table , etc)

pretty_table(T; backend = Val(:latex), header = header, formatters=ft_round(4), alignment=:c)


Outcome,CL,CRA,IRA,IRA W Lasso
0.0,-0.0855,-0.0797,-0.0755,0.0
0.0,0.0358,0.0356,0.0361,0.0
0.0,-0.1557,-0.1496,-0.1462,0.0
0.0,-0.0152,-0.0098,-0.0048,0.0


\begin{table}
  \begin{tabular}{ccccc}
    \hline\hline
    \textbf{Outcome} & \textbf{CL} & \textbf{CRA} & \textbf{IRA} & \textbf{IRA W Lasso} \\\hline
    Estimation & -0.0855 & -0.0797 & -0.0755 & 0.0 \\
    Standar error & 0.0358 & 0.0356 & 0.0361 & 0.0 \\
    Lower bound CI & -0.1557 & -0.1496 & -0.1462 & 0.0 \\
    Upper bound CI & -0.0152 & -0.0098 & -0.0048 & 0.0 \\\hline\hline
  \end{tabular}
\end{table}


# Using HDMJL

In [ ]:
include("../hdmjl/hdmjl.jl")

In [ ]:
rlasso = rlasso_arg( X, Y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

In [ ]:
rlasso_ira = rlasso(rlasso)